In [1]:
import os
import pandas as pd
from sqlalchemy import create_engine, text
from dotenv import load_dotenv
from datetime import datetime

In [ ]:
load_dotenv()
engine = create_engine(os.getenv("DATABASE_URL"))

with engine.begin() as conn:
    src_df = pd.read_sql("SELECT * FROM stg.stg_customers", conn)
    tgt_df = pd.read_sql("SELECT customer_code, customer_name, customer_type, address_line1, address_line2, city, state, postal_code, country, phone_number, email, region, territory, process_ind FROM int.customers", conn)

    src_df['process_ind'] = 'I'
    src_df['audit_insrt_dt'] = datetime.now()
    src_df['audit_updt_dt'] = datetime.now()
    src_df['audit_insrt_nm'] = None
    src_df['audit_updt_nm'] = None
    src_df['audit_batch_id'] = None

    merged_df = src_df.merge(tgt_df, on='customer_code', how='outer', indicator=True, suffixes=('_src', '_tgt'))

    new_rows = merged_df[merged_df['_merge'] == 'left_only']
    updated_rows = merged_df[merged_df['_merge'] == 'both']
    deleted_rows = merged_df[merged_df['_merge'] == 'right_only']

    if not new_rows.empty:
        new_rows = new_rows.rename(columns={col: col.replace('_src', '') for col in new_rows.columns if col.endswith('_src')})
        new_rows = new_rows[['customer_code', 'customer_name', 'customer_type', 'address_line1', 'address_line2', 'city', 'state', 'postal_code', 'country', 'phone_number', 'email', 'region', 'territory', 'process_ind', 'audit_insrt_dt', 'audit_insrt_nm', 'audit_updt_dt', 'audit_updt_nm', 'audit_batch_id']]

In [ ]:
new_rows

In [ ]:
updated_rows

In [ ]:
if not new_rows.empty:
            new_rows = new_rows.rename(columns={col: col.replace('_src', '') for col in new_rows.columns if col.endswith('_src')})
            new_rows = new_rows[['customer_code', 'customer_name', 'customer_type', 'address_line1', 'address_line2', 'city', 'state', 'postal_code', 'country', 'phone_number', 'email', 'region', 'territory', 'process_ind', 'audit_insrt_dt', 'audit_insrt_nm', 'audit_updt_dt', 'audit_updt_nm', 'audit_batch_id']]

In [ ]:
with engine.begin() as conn:
        src_df = pd.read_sql("SELECT * FROM stg.stg_customers", conn)
        tgt_df = pd.read_sql("SELECT customer_code, customer_name, customer_type, address_line1, address_line2, city, state, postal_code, country, phone_number, email, region, territory, process_ind FROM int.customers", conn)

        # Normalize data types
        src_df['customer_code'] = src_df['customer_code'].str.strip()
        tgt_df['customer_code'] = tgt_df['customer_code'].str.strip()

        # Merge to find new, updated, and deleted records
        merged_df = src_df.merge(tgt_df, on='customer_code', how='outer', indicator=True, suffixes=('_src', '_tgt'))

In [ ]:
merged_df['process_ind'] = 'I'  # Default to Insert
merged_df.loc[merged_df['_merge'] == 'both', 'process_ind'] = 'U'  # Update
merged_df.loc[merged_df['_merge'] == 'right_only', 'process_ind'] = 'D'  # Deleted

In [ ]:
insert_df = merged_df[merged_df['process_ind'] == 'I'].copy()
update_df = merged_df[merged_df['process_ind'] == 'U'].copy()


In [ ]:
insert_df 


In [7]:
from langchain_groq import ChatGroq
from langchain.schema import SystemMessage, HumanMessage
llm = ChatGroq(
    model="llama-3.1-70b-versatile",  # or "llama-3.2-90b-text-preview" for better reasoning
    temperature=0.2,
)





In [ ]:
completion = client.chat.completions.create(
  model="deepseek/deepseek-chat-v3.1:free",
  messages=[
    {
      "role": "user",
      "content": "What is the meaning of life?"
    }
  ]
)
print(completion.choices[0].message.content)

In [3]:
from openai import OpenAI

client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key="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"
)

resp = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Say hello from OpenRouter!"}
    ]
)

print(resp.choices[0].message.content)


APIConnectionError: Connection error.

In [4]:
import requests
r = requests.get("https://openrouter.ai/api/v1/models")
print(r.status_code, r.text[:200])


SSLError: HTTPSConnectionPool(host='openrouter.ai', port=443): Max retries exceeded with url: /api/v1/models (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1028)')))

In [5]:
import ssl, certifi, requests

print("SSL Cert File:", certifi.where())

# test HTTPS verification using the certifi CA bundle
r = requests.get("https://openrouter.ai/api/v1/models", verify=certifi.where())
print("Response:", r.status_code)


SSL Cert File: c:\Users\A4627\AppData\Local\Programs\Python\Python313\Lib\site-packages\certifi\cacert.pem


SSLError: HTTPSConnectionPool(host='openrouter.ai', port=443): Max retries exceeded with url: /api/v1/models (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1028)')))

In [6]:
python -m ssl


SyntaxError: invalid syntax (37222790.py, line 1)